In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.applications import EfficientNetB0
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Define paths
train_data_dir = '/home/vipinainvijayan4286/Data/train'
test_data_dir = '/home/vipinainvijayan4286/Data/test'

In [3]:
# Define image parameters
img_height, img_width = 229, 229
batch_size = 32

In [4]:
# Create data generators with augmentation for training and without augmentation for testing
train_datagen = ImageDataGenerator(validation_split = 0.20,rotation_range=20,)

In [5]:
test_datagen = ImageDataGenerator(validation_split = 0.20,rotation_range=20,)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = True,subset='training',seed = 42
)

Found 4116 images belonging to 3 classes.


In [7]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = True,subset='training',seed = 42
)

Found 1031 images belonging to 3 classes.


In [8]:
# Load pre-trained EfficientNetB0 model without top layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

In [9]:
# Freeze the pre-trained layers
base_model.trainable = False

# Build your custom model on top of the pre-trained base model
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))  # 3 output classes

In [10]:
# Compile the model
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 8, 8, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               327936    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 4,378,278
Trainable params: 328,707
Non-trainable params: 4,049,571
_________________________________________________________________


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    verbose=1,
    validation_data=test_generator
)

Epoch 1/10
129/129 [==============================] - 1166s 9s/step - loss: 0.3180 - accuracy: 0.8732 - val_loss: 0.1799 - val_accuracy: 0.9331
Epoch 2/10
129/129 [==============================] - 1280s 10s/step - loss: 0.1849 - accuracy: 0.9264 - val_loss: 0.1851 - val_accuracy: 0.9379
Epoch 3/10
129/129 [==============================] - 1231s 10s/step - loss: 0.1790 - accuracy: 0.9300 - val_loss: 0.1681 - val_accuracy: 0.9428
Epoch 4/10
129/129 [==============================] - 1116s 9s/step - loss: 0.1592 - accuracy: 0.9383 - val_loss: 0.1777 - val_accuracy: 0.9350
Epoch 5/10
129/129 [==============================] - 1137s 9s/step - loss: 0.1496 - accuracy: 0.9448 - val_loss: 0.1708 - val_accuracy: 0.9476
Epoch 6/10
129/129 [==============================] - 1218s 9s/step - loss: 0.1400 - accuracy: 0.9441 - val_loss: 0.1708 - val_accuracy: 0.9370
Epoch 7/10
 27/129 [=====>........................] - ETA: 13:49 - loss: 0.1592 - accuracy: 0.9387

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test Accuracy:", accuracy)

In [24]:
model.save('/home/vipinainvijayan4286/ChestXray-Classification-App/model.h5')

/usr/local/anaconda/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [34]:
# code for predicting an image stored locally against a trained model
# my local image is 28 x 28 already
import numpy as np
from PIL import Image
from keras.preprocessing import image
img = image.load_img('/home/vipinainvijayan4286/ChestXray-Classification-App/uploads/P.jpg',target_size = (229, 229))# , target_size=(32,32))
img  = image.img_to_array(img)
img  = img.reshape((1,) + img.shape)
# img  = img/255
#img = img.reshape(-1,229, 229,3)
img_class=model.predict(img) 
# this model above was already trained 
# code from https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-#neural-networks-python-keras/
prediction = img_class[0]
classname=list(test_generator.class_indices.keys())
print(classname[np.argmax(prediction, axis=0)])

PNEUMONIA


In [ ]:
# Extracting training and validation metrics from the history object
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(train_acc) + 1)

    # Plotting accuracy
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_acc, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plotting loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
test_generator = test_datagen.flow_from_directory(test_data_dir,target_size=(img_height, img_width),batch_size=32,shuffle=False,)


In [ ]:
predictions=model.predict(test_generator)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt


# Get true labels
true_labels = test_generator.classes

# Use the model to predict the classes
predicted_labels = np.argmax(model.predict(test_generator), axis=1)

# Generate confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Display confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Display classification report
print("Classification Report:\n", classification_report(true_labels, predicted_labels, target_names=test_generator.class_indices.keys()))